In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = os.path.join(os.getcwd(), '..', 'transformers')
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import pfrl
from textrl import TextRLEnv, TextRLActor, train_agent_with_evaluation
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import logging
import sys

/mnt/hdd1/venkat/RL/Text_Generation/venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

checkpoint = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model.eval()
model.cuda()

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.



T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [4]:
sentiment = pipeline('sentiment-analysis',model="cardiffnlp/twitter-roberta-base-sentiment",tokenizer="cardiffnlp/twitter-roberta-base-sentiment",device=0,return_all_scores=True)

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


/mnt/hdd1/venkat/RL/Text_Generation/venv/lib/python3.7/site-packages/transformers/pipelines/text_classification.py:107: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  UserWarning,


In [12]:
class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
      reward = 0
      if finish or len(predicted_list[0]) >= self.env_max_length:
        predicted_text = tokenizer.convert_tokens_to_string(predicted_list[0])
        # sentiment classifier
        reward = sentiment(input_item['input']+predicted_text)[0][0]['score'] * 10
      return [reward]

In [13]:
observation_list = [{"input": "Dogecoin is "}]
env = MyRLEnv(
    model, tokenizer, 
    observation_input=observation_list, 
    max_length=25, 
    compare_sample=1
)
actor = TextRLActor(
    env, model, tokenizer, 
    act_deterministically=False, 
    optimizer='adamw',
    temperature=0.8,
    top_k=100,
    top_p=0.85
)
agent = actor.agent_ppo(
    update_interval=2, 
    minibatch_size=2, 
    epochs=10,
    lr=3e-4
)

In [14]:
test_phrase = "Dogecoin is"
print(test_phrase)
actor.predict({'input': test_phrase})

Dogecoin is


['Dogecoin is a Bitcoin coin.</s>']

In [15]:
train_agent_with_evaluation(
    agent,
    env,
    steps=3000,
    eval_n_steps=None,
    eval_n_episodes=1,       
    train_max_episode_len=100,  
    eval_interval=10,
    outdir='checkpoint', 
)

/mnt/hdd1/venkat/RL/Text_Generation/venv/lib/python3.7/site-packages/pfrl/agents/ppo.py:133: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  actions = torch.tensor([b["action"] for b in dataset], device=device)


outdir:checkpoint step:10 episode:0 R:0.01742454362101853
statistics:[('average_value', -0.0012292142), ('average_entropy', 0.6083449), ('average_value_loss', 0.0014852832090718948), ('average_policy_loss', -0.0384124094247818), ('n_updates', 50), ('explained_variance', -3495.8016238104983)]
evaluation episode 0 length:11 R:0.11191997677087784
The best score is updated -3.4028235e+38 -> 0.11191997677087784
Saved the agent to checkpoint/best
outdir:checkpoint step:29 episode:1 R:0.417829304933548
statistics:[('average_value', 0.022493685), ('average_entropy', 0.8823873), ('average_value_loss', 0.0015162564466481853), ('average_policy_loss', -0.08375232934951782), ('n_updates', 140), ('explained_variance', -33.689571706558105)]
evaluation episode 0 length:27 R:0.22952584084123373
The best score is updated 0.11191997677087784 -> 0.22952584084123373
Saved the agent to checkpoint/best
outdir:checkpoint step:44 episode:2 R:0.23899082094430923
statistics:[('average_value', 0.035304867), ('ave

/mnt/hdd1/venkat/RL/Text_Generation/venv/lib/python3.7/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


Saved the agent to checkpoint/best
outdir:checkpoint step:71 episode:3 R:0.1855374127626419
statistics:[('average_value', 0.05876245), ('average_entropy', 0.88697404), ('average_value_loss', 0.0012910083118515558), ('average_policy_loss', -0.071583311855793), ('n_updates', 350), ('explained_variance', 0.9638701779350602)]
evaluation episode 0 length:27 R:0.3583761677145958
outdir:checkpoint step:91 episode:4 R:0.3069256618618965
statistics:[('average_value', 0.06962408), ('average_entropy', 0.89410406), ('average_value_loss', 0.0006044512026983284), ('average_policy_loss', -0.05750807166099548), ('n_updates', 450), ('explained_variance', -30.521068142752338)]
evaluation episode 0 length:27 R:0.2242595562711358
outdir:checkpoint step:118 episode:5 R:0.5087452940642834
statistics:[('average_value', 0.08708309), ('average_entropy', 0.8681084), ('average_value_loss', 0.0023938206943466865), ('average_policy_loss', -0.08393447160720825), ('n_updates', 590), ('explained_variance', 0.93629813

(<textrl.actor.TextPPO at 0x7fc94adcd860>,
 [{'average_value': -0.0012292142,
   'average_entropy': 0.6083449,
   'average_value_loss': 0.0014852832090718948,
   'average_policy_loss': -0.0384124094247818,
   'n_updates': 50,
   'explained_variance': -3495.8016238104983,
   'eval_score': 0.11191997677087784},
  {'average_value': 0.022493685,
   'average_entropy': 0.8823873,
   'average_value_loss': 0.0015162564466481853,
   'average_policy_loss': -0.08375232934951782,
   'n_updates': 140,
   'explained_variance': -33.689571706558105,
   'eval_score': 0.22952584084123373},
  {'average_value': 0.035304867,
   'average_entropy': 0.9291828,
   'average_value_loss': 0.0061693967462997535,
   'average_policy_loss': -0.09065193623304367,
   'n_updates': 220,
   'explained_variance': -47.999845285517935,
   'eval_score': 0.43175079859793186},
  {'average_value': 0.05876245,
   'average_entropy': 0.88697404,
   'average_value_loss': 0.0012910083118515558,
   'average_policy_loss': -0.0715833118

In [16]:
agent.load('./checkpoint/best')
print(observation_list[0])
actor.predict(observation_list[0])

{'input': 'Dogecoin is '}


['dog</s>']